In [2]:
import os
import cv2
import numpy as np
from scipy.linalg import eigh
from sklearn.cluster import KMeans
from traditional_utils import *

In [3]:
# Read images from folders A and B
imagesA = []
imagesB = []
labels = []

for i in range(4868):
    if os.path.exists('./dataset/train/A/'+str(i).zfill(4)+'.png'):
        imgA = cv2.imread('./dataset/train/A/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        # imgG = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
        
        imagesA.append(imgA)

print(len(imagesA))

3895


In [4]:
for i in range(4868):
    if os.path.exists('./dataset/train/B/'+str(i).zfill(4)+'.png'):
        imgB = cv2.imread('./dataset/train/B/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        # imgG = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)
        
        imagesB.append(imgB)

print(len(imagesB))

3895


In [5]:
for i in range(4868):
    if os.path.exists('./dataset/train/OUT/'+str(i).zfill(4)+'.png'):
        label = cv2.imread('./dataset/train/OUT/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        # imgG = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        
        labels.append(label)

print(len(labels))

3895


In [43]:
def pca_kmeans(imgA, imgB, block_size, rate=0.9):

    # Calculate the image size and padding size
    image_size = imgA.shape
    padding_size = np.array(image_size) + block_size
    padding_size[2] = padding_size[2] - block_size

    # Calculate the difference image
    diff = imgB.astype(float) - imgA.astype(float)

    # Padding
    padding_img = np.zeros(padding_size, dtype=diff.dtype)
    ub_col = block_size // 2 + image_size[0] - 1
    ub_row = block_size // 2 + image_size[1] - 1
    padding_img[block_size // 2:ub_col + 1, block_size // 2:ub_row + 1, :] = diff

    # Generate feature vector for blocks
    feature_vector = np.zeros((image_size[0] * image_size[1], image_size[2] * block_size * block_size))
    for k1 in range(image_size[0]):
        for k2 in range(image_size[1]):
            temp_feature_vector = padding_img[k1:k1 + block_size, k2:k2 + block_size, :]
            feature_vector[k1 * image_size[1] + k2, :] = temp_feature_vector.reshape(-1)

    # Normalization
    mean_val = np.mean(feature_vector, axis=0)
    std_val = np.std(feature_vector, axis=0) + 1e-12
    feature_vector = (feature_vector - mean_val) / std_val

    # PCA
    cov = np.dot(feature_vector.T, feature_vector) # Covariance matrix
    val, vec = eigh(cov) # Eigenvalues and eigenvectors
    val = val[::-1] # Reverse the order
    vec = vec[:, ::-1] # Reverse the order
    for k1 in range(len(val)):
        if sum(val[k1:]) >= rate * sum(val): # Select the number of principal components
            break
    vec = vec[:, k1:] # Select the principal components
    feature = np.dot(feature_vector, vec) # Feature vector after PCA

    # k-means
    kmeans = KMeans(n_clusters=2) # 2 clusters
    label = kmeans.fit_predict(feature) # Cluster labels
    change_map = label.reshape(image_size[1], image_size[0]).T # Change map
    if len(change_map[change_map == 0]) > len(change_map[change_map == 1]): # Determine the label of the change area (Assuming most aren't changed)
        change_map[change_map == 0] = 0 
        change_map[change_map == 1] = 255
    else:
        change_map[change_map == 0] = 255
        change_map[change_map == 1] = 0

    return change_map

In [44]:
diff = pca_kmeans(imagesA[0], imagesB[0], 5)
print(diff.shape)

(256, 256)


In [45]:
import matplotlib.pyplot as plt
# convert diff to be 0 or 1
diff = diff.astype(np.uint8)
np.unique(diff)

cv2.imshow('diff', diff)
cv2.waitKey(0)
cv2.destroyAllWindows()

diff[diff == 0] = 0
diff[diff == 255] = 1



In [32]:
# Calculate thejacard index
label = labels[0]
label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)

label[label == 0] = 0
label[label == 255] = 1

jaccard_index =  compute_iou(diff, label)
print(jaccard_index)

0.004675984752223634


In [46]:
def detect_edges(image):

    image = image.astype(np.uint8)

    # Step 2: Preprocess the Image
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 3: Edge Detection
    edges = cv2.Canny(blurred, 50, 150)

    # Step 4: Find Contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the original image to draw contours
    contour_image = image.copy()

    # Step 5: Filter Contours and Draw
    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Filter based on the number of sides (rectangles and circular shapes)
        if len(approx) >= 4:
            # Draw the contour on the image
            cv2.drawContours(contour_image, [approx], -1, (0, 255, 0), 2)
            
    return contour_image

In [47]:
jaccard = 0
i = 0
for imgA, imgB, label in zip(imagesA, imagesB, labels):
    diff = pca_kmeans(imgA, imgB, 5)
    diff = detect_edges(diff)
    diff = diff.astype(np.uint8)
    diff[diff == 0] = 0
    diff[diff == 255] = 1
    label[label == 0] = 0
    label[label == 255] = 1

    label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
    jaccard_index =  compute_iou(diff, label)
    jaccard += jaccard_index
    print(f'{i} ' if i % 100 == 0 else '', end='')
    i += 1
    
print(jaccard/len(imagesA))

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 0.040058448642159454


In [48]:
# Read images from folders A and B
valA = []
valB = []
valLabels = []

for i in range(4868):
    if os.path.exists('./dataset/val/A/'+str(i).zfill(4)+'.png'):
        imgA = cv2.imread('./dataset/val/A/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        # imgG = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
        
        valA.append(imgA)

print(len(valA))

973


In [49]:
for i in range(4868):
    if os.path.exists('./dataset/val/B/'+str(i).zfill(4)+'.png'):
        imgB = cv2.imread('./dataset/val/B/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        # imgG = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)
        
        valB.append(imgB)

print(len(valB))

973


In [50]:
for i in range(4868):
    if os.path.exists('./dataset/val/OUT/'+str(i).zfill(4)+'.png'):
        label = cv2.imread('./dataset/val/OUT/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        # imgG = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        
        valLabels.append(label)

print(len(valLabels))

973


In [52]:
jaccard = 0
i = 0
for imgA, imgB, label in zip(valA, valB, valLabels):
    diff = pca_kmeans(imgA, imgB, 5)
    diff = diff.astype(np.uint8)
    diff[diff == 0] = 0
    diff[diff == 255] = 1
    label[label == 0] = 0
    label[label == 255] = 1

    label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
    jaccard_index =  compute_iou(diff, label)
    jaccard += jaccard_index
    print(f'{i} ' if i % 100 == 0 else '', end='')
    i += 1
    
print(jaccard/len(imagesA))

0 100 200 300 400 500 600 700 800 900 0.01110049140609563


In [53]:
jaccard = 0
i = 0
for imgA, imgB, label in zip(valA, valB, valLabels):
    diff = pca_kmeans(imgA, imgB, 5)
    diff = detect_edges(diff)
    diff = diff.astype(np.uint8)
    diff[diff == 0] = 0
    diff[diff == 255] = 1
    label[label == 0] = 0
    label[label == 255] = 1

    label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
    jaccard_index =  compute_iou(diff, label)
    jaccard += jaccard_index
    print(f'{i} ' if i % 100 == 0 else '', end='')
    i += 1
    
print(jaccard/len(imagesA))

0 100 200 300 400 500 600 700 800 900 0.00968862030786207
